# basic interface idea (prototype)

In [ ]:
import semicon

bands = ['gamma_6c', 'gamma_8v', 'gamma_7v']

# this will be probably a sympy model, that
# can directly be used by ``kwant.continuum.discretizer``.

hamiltonian = semicon.models.kane(
    components=['base', 'zeeman', 'dresselhaus'], 
    bands=bands,
    coords='z',
)


# this will be a dictionary that can be passed into ``params`` of kwant system.
# possible it will also have attributes with extra information about material
# sources, etc.

parameters = semicon.parameters.two_deg(
    bank='lawaetz',
    materials=['AlSb', 'InAs', 'GaSb', 'InAs'],
    widths=[5, 12.5, 5, 5],
    valence_band_offsets=[.18, .0, .56, .18]
    extra_constants={'hbar': 1, 'e': 1},
    bands=bands,
)


shape = semicon.shapes.two_deg(L=sum(parameters.widths))

In [ ]:
import kwant
template = kwant.continuum.discretize(hamiltonian, coords='z', grid_spacing=0.5)
syst = kwant.Builder()

syst.fill(template, shape, (0,))
syst = syst.finalized()

ham = syst.hamiltonian_submatrix(params=parameters)